In [35]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import numpy as np
import os
from osgeo import gdal

In [ ]:
# Set which GPU this process can see
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# Paths to training/validation imagery and labels
# Each sample image is a multi-band .tif tile
# Each label is a single-band .tif
# Image and label files share the same filename and live in parallel folders
train_data_dir = 'your own link'
train_label_dir = 'your own link'
validation_data_dir = 'your own link'
validation_label_dir = 'your own link'

# Data loading & preprocessing
# Assumes every image in `data_dir` has a corresponding label with the same filename in `label_dir`.
def load_and_preprocess_data(data_dir, label_dir):
    image_list = []
    label_list = []

    for image_file in os.listdir(data_dir):
        if image_file.endswith(".tif"):
            image_path = os.path.join(data_dir, image_file)
            label_path = os.path.join(label_dir, image_file) 

            image_ds = gdal.Open(image_path, gdal.GA_ReadOnly)
            label_ds = gdal.Open(label_path, gdal.GA_ReadOnly)

            image = image_ds.ReadAsArray()
            label = label_ds.ReadAsArray()


            image_list.append(image)
            label_list.append(label)

            image_ds = None
            label_ds = None

    image_array = np.array(image_list)
    label_array = np.array(label_list)

    return image_array, label_array


# Loading training and validation set data.
X_train, y_train = load_and_preprocess_data(train_data_dir, train_label_dir)
X_val, y_val = load_and_preprocess_data(validation_data_dir, validation_label_dir)

# One-hot encode labels to 6 classes
y_train_one_hot = tf.one_hot(y_train, 6)
y_val_one_hot = tf.one_hot(y_val, 6)

# Reorder image dimensions from (N, C, H, W) to (N, H, W, C)
X_train = X_train.transpose(0, 2, 3, 1)
X_val = X_val.transpose(0, 2, 3, 1)

# Normalize imagery
X_train_normalized = X_train.astype('float32') / 12000
X_val_normalized = X_val.astype('float32') / 12000

# checks: print shapes
print("Training data shape:", X_train_normalized.shape)
print("Training labels shape:", y_train_one_hot.shape)
print("Validation data shape:", X_val_normalized.shape)
print("Validation labels shape:", y_val_one_hot.shape)

In [37]:
class SaveBestModelAfterEpoch(Callback):
    
    # Keras callback to start tracking a validation metric from a given epoch and save the whole model whenever a new best score is observed.
    def __init__(self, filepath, start_epoch=15, monitor='val_accuracy', mode='max'):
        super(SaveBestModelAfterEpoch, self).__init__()
        self.filepath = filepath
        self.start_epoch = start_epoch
        self.monitor = monitor
        self.mode = mode
        
        # Initialize the "best" value depending on optimization direction
        self.best = -float('inf') if mode == 'max' else float('inf')
        
        # Placeholder if you later want to also keep best weights in-memory
        self.best_weights = None

    # At the end of each epoch, check the monitored metric in `logs`.If epoch` >= `start_epoch`and performance improves, save the model.
    def on_epoch_end(self, epoch, logs=None):
        
        # Only start tracking/saving after the warm-up period.
        if epoch >= self.start_epoch:  =
            current = logs.get(self.monitor)
            if self.mode == 'max' and current > self.best:
                self.best = current
                self.model.save(self.filepath, overwrite=True) # Save entire model; overwrite existing best.
                print(f'\nSaving new best model to {self.filepath}')
            elif self.mode == 'min' and current < self.best:
                self.best = current
                self.model.save(self.filepath, overwrite=True)
                print(f'\nSaving new best model to {self.filepath}')

In [ ]:
# U-Net encoder–decoder for 2D semantic segmentation
input_layer = Input(shape=(128, 128, 12))  

# Encoder (downsampling path)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

# Decoder (upsampling path)
up5 = UpSampling2D(size=(2, 2))(pool4)
up5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
merge5 = concatenate([conv4, up5], axis=-1)
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge5)

up6 = UpSampling2D(size=(2, 2))(conv5)
up6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
merge6 = concatenate([conv3, up6], axis=-1)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge6)

up7 = UpSampling2D(size=(2, 2))(conv6)
up7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
merge7 = concatenate([conv2, up7], axis=-1)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge7)

up8 = UpSampling2D(size=(2, 2))(conv7)
up8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
merge8 = concatenate([conv1, up8], axis=-1)
output_layer = Conv2D(6, (1, 1), activation='softmax')(merge8) 

model = Model(inputs=input_layer, outputs=output_layer)

# save the whole model after epoch >=15 whenever monitored metric improves
checkpoint_best_after_15 = SaveBestModelAfterEpoch('your own link.h5')

# Compile & Train
with tf.device('/GPU:2'):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(
        X_train_normalized, 
        y_train_one_hot, 
        validation_data=(X_val_normalized, y_val_one_hot), 
        epochs=30,
        callbacks=[checkpoint_best_after_15],
        batch_size=16
    )

# Evaluate on validation set
results = model.evaluate(X_val_normalized, y_val_one_hot)
print('Validation loss:', results[0])
print('Validation accuracy:', results[1])

In [ ]:
best_model = load_model('your own link.h5')

# Using the model to make predictions on the validation set
y_pred = best_model.predict(X_val_normalized)


# Convert the model's output into class labels
y_pred_labels = np.argmax(y_pred, axis=-1)
y_true_labels = np.argmax(y_val_one_hot, axis=-1)

print(y_pred_labels.shape)
print(y_true_labels.shape)

# Flatten to 1D vectors so sklearn metrics treat each pixel as an instance
y_pred_labels = y_pred_labels.reshape(-1)
y_true_labels = y_true_labels.reshape(-1)

# Calculate and display the confusion matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)
print(conf_matrix)